In [4]:
let GROQ_API_KEY = Deno.env.get("GROQ_API_KEY");
if (!GROQ_API_KEY) {
  const input = typeof prompt === "function" ? prompt("Enter GROQ_API_KEY:") : null;
  if (input) {
    Deno.env.set("GROQ_API_KEY", input);
    GROQ_API_KEY = input;
    // Tidak menampilkan API key ke console
  } else {
    throw new Error("GROQ_API_KEY not provided. Set it in the environment or provide it interactively.");
  }
}

## Using LLM Model in Langchain

In [11]:
import { ChatGroq } from "@langchain/groq";

const model = new ChatGroq({model: "llama-3.3-70b-versatile"});

const response = await model.invoke('Tell me a joke');

// console.log(response);
console.log(response.content);

Here's one:

What do you call a fake noodle?

An impasta!

Hope that made you laugh! Do you want to hear another one?


In [ ]:
import { ChatGroq } from "@langchain/groq";
import { HumanMessage, SystemMessage } from '@langchain/core/messages';

const model = new ChatGroq({model: "llama-3.3-70b-versatile"});
const prompt = [
  new SystemMessage(
    'You are a helpful assistant that responds to questions with three exclamation marks.'
  ),
  new HumanMessage('What is the capital of France?'),
];

const response = await model.invoke(prompt);

console.log(response.content);

The capital of France is Paris.


`HumanMessage`
A message sent from the perspective of the human, with the user role

`AIMessage`
A message sent from the perspective of the AI that the human is interacting with, with the assistant role

`SystemMessage`
A message setting the instructions the AI should follow, with the system role

`ChatMessage`
A message allowing for arbitrary setting of role

## Making LLM Prompts Reusable

In [13]:
import { PromptTemplate } from '@langchain/core/prompts';

const template =
  PromptTemplate.fromTemplate(`Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don't know".

Context: {context}

Question: {question}

Answer: `);

const response = await template.invoke({
  context:
    "The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's `transformers` library, or by utilizing OpenAI and Cohere's offerings through the `openai` and `cohere` libraries, respectively.",
  question: 'Which model providers offer LLMs?',
});

console.log(response);

StringPromptValue {
  lc_serializable: true,
  lc_kwargs: {
    value: `Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don't know".\n` +
      "\n" +
      "Context: The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's `transformers` library, or by utilizing OpenAI and Cohere's offerings through the `openai` and `cohere` libraries, respectively.\n" +
      "\n" +
      "Question: Which model providers offer LLMs?\n" +
      "\n" +
      "Answer: "
  },
  lc_namespace: [ "langchain_core", "prompt_values" ],
  value: `Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don't know".\n` +
    "\n"

In [15]:
import { PromptTemplate } from '@langchain/core/prompts';
import { ChatGroq } from "@langchain/groq";

const model = new ChatGroq({model: "llama-3.3-70b-versatile"});

const template =
  PromptTemplate.fromTemplate(`Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don't know".

Context: {context}

Question: {question}

Answer: `);

const prompt = await template.invoke({
  context:
    "The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's `transformers` library, or by utilizing OpenAI and Cohere's offerings through the `openai` and `cohere` libraries, respectively.",
  question: 'Which model providers offer LLMs?',
});

const response = await model.invoke(prompt);
console.log(response.content);

OpenAI and Cohere are mentioned as model providers that offer LLMs, in addition to Hugging Face.


In [16]:
import { ChatPromptTemplate } from '@langchain/core/prompts';
import { ChatGroq } from "@langchain/groq";

const model = new ChatGroq({model: "llama-3.3-70b-versatile"});

const template = ChatPromptTemplate.fromMessages([
  [
    'system',
    'Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don\'t know".',
  ],
  ['human', 'Context: {context}'],
  ['human', 'Question: {question}'],
]);

const prompt = await template.invoke({
  context:
    "The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's `transformers` library, or by utilizing OpenAI and Cohere's offerings through the `openai` and `cohere` libraries, respectively.",
  question: 'Which model providers offer LLMs?',
});

const response = await model.invoke(prompt);
console.log(response.content);

OpenAI and Cohere are mentioned as model providers that offer LLMs, alongside Hugging Face, which provides access to these models through its `transformers` library.


Prompt community

https://smith.langchain.com/hub/

In [ ]:
// Set the LANGSMITH_API_KEY environment variable (create key in Settings > API Keys)

// If you are in a non-Node environment, please use the default "langchain/hub" entrypoint and omit includeModel for providers other than OpenAI
import * as hub from "langchain/hub/node";

await hub.pull("hardkothari/prompt-maker", {
  includeModel: true
});

## Specific Formats out of LLMs

In [ ]:
import { ChatGroq } from "@langchain/groq";
import { z } from "zod";

const answerSchema = z
  .object({
    answer: z.string().describe("The answer to the user's question"),
    justification: z.string().describe(`Justification for the 
      answer`),
  })
  .describe(`An answer to the user's question along with justification for 
    the answer.`);


const model = new ChatGroq({model: "llama-3.3-70b-versatile"}).withStructuredOutput(answerSchema)

await model.invoke("What weighs more, a pound of bricks or a pound of feathers")

## Runnable Interface

There is a common interface with these methods:

`invoke`: transforms a single input into an output

`batch`: efficiently transforms multiple inputs into multiple outputs

`stream`: streams output from a single input as it’s produced

In [18]:
import { ChatGroq } from "@langchain/groq";

const model = new ChatGroq({model: "llama-3.3-70b-versatile"});

const response = await model.invoke('Hi there!');
console.log(response.content);
// Hi!

const completions = await model.batch(['Hi there!', 'Bye!']);
// ['Hi!', 'See you!']

for await (const token of await model.stream('Bye!')) {
  console.log(token.content);
  // Good
  // bye
  // !
}

It's nice to meet you. Is there something I can help you with or would you like to chat?

By
e
!
 It

By
e
!
 It
 was
 nice
 chatting
 with
 you
.
 Have
 a
 great
 day
!


 was
 nice
 chatting
 with
 you
.
 Have
 a
 great
 day
!


